## Dokumentation & Einleitung

# 🚀 Case Study: Technical Automation & AI Specialist
## Projekt: Automatisierter Workflow für Ad Creative Testing

---
**Autor:** Miroslav Jovanovic  
**Datum:** 01. Januar 2026  
**Status:** Final Submission (incl. API Error Handling)
---

### 1. Strategische Einleitung
Dieser Workflow automatisiert die Skalierung von Werbemitteln. Aus wenigen Basis-Headlines und Bild-Ideen wird eine umfangreiche Test-Matrix erstellt.

**Besonderheit dieser Version:**
- **Robustheit:** Enthält einen Fallback-Mechanismus (Mocking), falls die API das Quota-Limit erreicht.
- **Security:** API-Key wird sicher aus einer `.env`-Datei geladen.
- **Skalierbarkeit:** Generiert automatisch ein Cross-Product aller Assets.

## Setup & Key-Management (Code)

In [ ]:
# Installation der Pakete falls nötig:
# !pip install openai pandas python-dotenv

import openai
import pandas as pd
import json
import os
from dotenv import load_dotenv

# Lädt den Key aus deiner OPENAI_API_KEY.env
load_dotenv(dotenv_path="OPENAI_API_KEY.env")
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("❌ Fehler: Kein API-Key gefunden!")
else:
    client = openai.OpenAI(api_key=api_key)
    print(f"✅ API-Key sicher geladen ({api_key[:7]}...).")

## Datenbasis (Code)

In [ ]:
headlines = [
    "Sichere dir 0€ Anschaffungskosten für deine Solaranlage.",
    "Warum deine Rentenversicherung dich Geld kostet.",
    "Solarstrom auch bei Bewölkung? So geht's.",
    "Vermögensaufbau für den Mittelstand: Strategien 2026.",
    "E-Auto kostenlos laden mit eigener PV-Anlage."
]

image_prompts = [
    "Modernes Einfamilienhaus mit glänzenden Solarpanels auf dem Dach.",
    "Ein seriöser Finanzberater zeigt einem jungen Paar ein Tablet.",
    "Nahaufnahme eines Stromzählers, der rückwärts läuft."
]

## KI-Verarbeitung mit Fallback-Logik (Code)

In [ ]:
def process_ai_assets(headlines, prompts):
    print("🤖 AI-Verarbeitung gestartet...")
    opt_headlines = []
    opt_prompts = []

    try:
        # Versuch der echten API-Optimierung
        for h in headlines:
            res = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "Erstelle zwei Varianten (v1: Emotional, v2: Benefit) als JSON: {'v1': '...', 'v2': '...'}"},
                    {"role": "user", "content": h}
                ],
                response_format={ "type": "json_object" }
            )
            opt_headlines.append(json.loads(res.choices[0].message.content))

        for p in prompts:
            res = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "system", "content": "Optimiere für DALL-E 3 (fotorealistisch)."}, {"role": "user", "content": p}]
            )
            opt_prompts.append(res.choices[0].message.content)
            
        print("✨ KI-Optimierung erfolgreich.")

    except Exception as e:
        print(f"⚠️ API-Quota (429) oder Fehler erkannt. Nutze Simulations-Modus.")
        # Simulation der KI-Ergebnisse (Mocking), damit der Workflow weiterläuft
        for h in headlines:
            opt_headlines.append({"v1": f"🔥 [KI-Emotional] {h}", "v2": f"✅ [KI-Benefit] {h}"})
        for p in prompts:
            opt_prompts.append(f"Optimierter High-End Prompt für: {p} (Cinematic, 8k)")
            
    return opt_headlines, opt_prompts

final_headlines, final_prompts = process_ai_assets(headlines, image_prompts)

## Matrix-Erstellung & Export (Code)

In [ ]:
matrix_data = []
counter = 1

for i, h_group in enumerate(final_headlines):
    for j, p_opt in enumerate(final_prompts):
        matrix_data.append({
            "Kombinations-ID": f"SET-{counter:03d}",
            "Headline_V1": h_group['v1'],
            "Headline_V2": h_group['v2'],
            "Bild_Prompt_Final": p_opt,
            "Status": "Ready for Review"
        })
        counter += 1

df = pd.DataFrame(matrix_data)
df.to_csv("ad_creative_matrix.csv", index=False, encoding='utf-8-sig')

print(f"✅ Matrix mit {len(df)} Zeilen erstellt.")
df.style.set_properties(**{'text-align': 'left'}) # Formatiert die Anzeige in VS Code schöner
df.head(10)

## Abschluss-Doku

### 4. Fazit & Skalierung
Dieser Workflow zeigt, wie technische Hürden (wie API-Limits) durch intelligentes Code-Design (Try-Except/Mocking) abgefangen werden können. 

**Nächste Schritte:**
- **Automatisierter Import:** Integration der Airtable-API für direkten Daten-Push.
- **Echtzeit-Assets:** Verknüpfung mit der DALL-E 3 API für sofortige Bildgenerierung.